In [1]:
import numpy as np 
import torch 
import torch.nn as nn
import torch.nn.functional as F 
from torch.utils.data import Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
from torchvision.io import read_image
from torch.utils.data import DataLoader
import torch.optim as optim
import os
from torchvision import datasets
from torch.utils.data import DataLoader, random_split
from torch.autograd import Variable

In [2]:
#Tensorboard framework
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/KMNIST')

In [ ]:
#References 
#[1] :
#https://medium.com/@nutanbhogendrasharma/pytorch-convolutional-neural-network-with-mnist-dataset-4e8a4265e118
#[2] : https://www.geeksforgeeks.org/training-neural-networks-with-validation-using-pytorch/
#[3] : https://nextjournal.com/gkoehler/pytorch-mnist

In [3]:
#Transforms object 

transform_comp = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5)),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.Pad(4, fill=0, padding_mode='constant'),
     transforms.RandomCrop((28,28), padding=None, pad_if_needed=False, fill=0, padding_mode='constant')
     ])

#Constant parameters for training & optimizer
batchsize = 4
epochs = 10
lr = 0.001

#Preprocessing the data by loading it, splitting into validation & training
train = datasets.KMNIST('data/', train = True, transform = transform_comp, download = True)
test = datasets.KMNIST('data/', train = False, transform = transform_comp, download = True)
train, valid = random_split(train,[59000,1000])
trainloader = DataLoader(train,batch_size = batchsize, shuffle=True)
testloader = DataLoader(test,batch_size = batchsize, shuffle=True)
valloader = DataLoader(valid,batch_size = batchsize, shuffle=True)

In [4]:
#CNN network with the entire architecture 
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=32,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(32, 64, 3, 1, 1),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        # fully connected layer, output 10 classes
        self.fully_connected = nn.Sequential(
            nn.Linear(64*7*7,1024),
            nn.ReLU(),
            nn.Linear(1024,10)
        )
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        # flatten the output of conv2 to (batch_size, 32 * 7 * 7)

        x = x.view(x.size(0), -1)       
        output = self.fully_connected(x)
        return output, x    # return x for visualization

In [5]:
#Creating an instance of the model 
cnn = CNN()

#Setting up the loss function & optimizer 
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), betas=(0.9, 0.999), lr=lr)

In [6]:
#Constant values for the training process & performance monitoring 
epochs = 30
min_valid_loss = np.inf
total_train = 0 
correct_train = 0
total_val = 0 
correct_val = 0

train_losses = []
train_accuracy = []
val_losses = []
val_accuracy = []

#Training loop 
for e in range(epochs):
    train_loss = 0.0
    #Total length of the loaded training data 
    total_step = len(trainloader)

    #Setting up the model object for training 
    cnn.train()     
    for i, (images, labels) in enumerate(trainloader):
        
        #Taking a single image and label from the loaded data
        b_x = Variable(images)   
        b_y = Variable(labels) 
        output = cnn(b_x)[0] 

        #Calculating the loss on the single output        
        loss = loss_func(output, b_y)

        #Reset the gradients 
        optimizer.zero_grad()

        #Calculate the derivate of the loss with respect to given parameters
        loss.backward()

        #Optimiser takes a step based on calculated gradient value 
        optimizer.step()
        
        #Write the loss to tensorboard 
        writer.add_scalar("Loss/train", loss.item(), e)

        #Calculate running loss & accuracy 
        train_loss += loss.item()
        train_losses.append(loss.item())
        if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(e + 1, epochs, i + 1, total_step, loss.item()))

        _, predicted = output.max(1)
        total_train += labels.size(0)
        correct_train += predicted.eq(labels).sum().item()
        accu_train=100.*correct_train/total_train
        writer.add_scalar("Accu/train", accu_train, e)
        train_accuracy.append(accu_train)

    
    #Similar process for validation data 
    valid_loss = 0.0
    cnn.eval()    
    for i, (images, labels) in enumerate(valloader):
        
        b_x = Variable(images)   
        b_y = Variable(labels)   
        output = cnn(b_x)[0]               
        loss = loss_func(output, b_y)
        writer.add_scalar("Loss/val", loss.item(), e)
        valid_loss += loss.item()
        val_losses.append(loss.item())

        _, predicted = output.max(1)
        total_val += labels.size(0)
        correct_val += predicted.eq(labels).sum().item()
        accu_val=100.*correct_val/total_val
        writer.add_scalar("Accu/val", accu_val, e)
        val_accuracy.append(accu_val)

    print(f'Epoch {e+1} \t\t Training Loss: {train_loss / len(trainloader)} \t\t Validation Loss: {valid_loss / len(valloader)}')
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
        # Saving State Dict
        torch.save(cnn.state_dict(), 'checkpoints/saved_model.pth')

Epoch [1/30], Step [100/14750], Loss: 2.0321
Epoch [1/30], Step [200/14750], Loss: 1.4519
Epoch [1/30], Step [300/14750], Loss: 1.4261
Epoch [1/30], Step [400/14750], Loss: 1.8854
Epoch [1/30], Step [500/14750], Loss: 1.8234
Epoch [1/30], Step [600/14750], Loss: 2.9770
Epoch [1/30], Step [700/14750], Loss: 0.9725
Epoch [1/30], Step [800/14750], Loss: 0.9993
Epoch [1/30], Step [900/14750], Loss: 0.7261
Epoch [1/30], Step [1000/14750], Loss: 1.0481
Epoch [1/30], Step [1100/14750], Loss: 2.3092
Epoch [1/30], Step [1200/14750], Loss: 0.9144
Epoch [1/30], Step [1300/14750], Loss: 1.4192
Epoch [1/30], Step [1400/14750], Loss: 1.9067
Epoch [1/30], Step [1500/14750], Loss: 1.4603
Epoch [1/30], Step [1600/14750], Loss: 0.3220
Epoch [1/30], Step [1700/14750], Loss: 1.3495
Epoch [1/30], Step [1800/14750], Loss: 1.8749
Epoch [1/30], Step [1900/14750], Loss: 0.8693
Epoch [1/30], Step [2000/14750], Loss: 0.4236
Epoch [1/30], Step [2100/14750], Loss: 0.3391
Epoch [1/30], Step [2200/14750], Loss: 0.35

In [7]:
def test():
    # Test the model
    cnn.eval()
    with torch.no_grad():
        count=0
        correct = 0
        total = 0
        for images, labels in testloader:
            
            #Calculate accuracy for each sample
            test_output, last_layer = cnn(images)
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            accuracy = (pred_y == labels).sum().item() / float(labels.size(0))

            #Write to tensorboard 
            writer.add_scalar("Accu/test", accuracy, count)
            count = count + 1
            pass

        print('Test Accuracy of the model on the 10000 test images: %.2f' % accuracy)
    
pass
test()

Test Accuracy of the model on the 10000 test images: 0.75
